In [13]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd
import time
import seaborn as sns
import sys
import tensorflow as tf

In [3]:
ds = []
for i in range(5):
  ind = np.load('data_dic_' + str(i+1) + '.npy',allow_pickle='TRUE').item()
  ds.append(ind)

print(f'Dataset is dowloaded and it is a list of size {len(ds)}')

Dataset is dowloaded and it is a list of size 5


# Select a random dataset plus Reading labels

In [4]:
seed_value = 21
np.random.seed(seed_value)
ds_num = np.random.randint(5)

LOS_labels = np.transpose(ds[ds_num]['LOS'][()][()][()],[1,0]) #first measurement campaign in Radian --> azimuth angle of arrivial (phi)
LOS = np.where(LOS_labels == 1.1, 0, 1)

x = ds[ds_num]['x'][()][()][()] #first measurement campaign in Radian --> azimuth angle of arrivial (phi)
y = ds[ds_num]['y'][()][()][()] #first measurement campaign in Radian --> azimuth angle of arrivial (phi)
z = ds[ds_num]['z'][()][()][()] #first measurement campaign in Radian --> azimuth angle of arrivial (phi)
tx_x= ds[ds_num]['tx_x'][()][()][()] #first measurement campaign in Radian --> azimuth angle of arrivial (phi)
tx_y = ds[ds_num]['tx_y'][()][()][()] #first measurement campaign in Radian --> azimuth angle of arrivial (phi)
tx_z = ds[ds_num]['tx_z'][()][()][()] #first measurement campaign in Radian --> azimuth angle of arrivial (phi)
pow = np.transpose(10**(0.1*(ds[ds_num]['pows'][()][()][()])), [1,0,2]) #first measurement campaign in Watt

print(f'Current random selected dataset is: {ds_num}')
#print(f'The shape of dataset of cartesian coordinate system is \n x = {x.shape} \n y = {y.shape} \n z = {z.shape} \n tx_x = {tx_x.shape} \n tx_y = {tx_y.shape} \n tx_z = {tx_z.shape}')
print(f'LOS/NLOS label counts: {np.unique(LOS[:,17], return_counts=True)}')

Current random selected dataset is: 1
LOS/NLOS label counts: (array([0, 1]), array([6756,  346]))


# PRSmaker class object

In [5]:
class PRSmaker:
  def __init__(self):
    self.output = None
    self.PNS = None
    self.PRS = None
    self.PRS_CP = None
    self.nFFT = None
    self.N_slot_symb = None
    self.n_s_f = None
    self.K_PRS_comb = None

  def PNSequence_5g(self, n_s_f, l,  Mpn, n_PRS_ID, N_slot_symb):

    """
    input Parameter Description:
        ns: time slot number
        n:_PRS_ID: slot_symb --- PRS sequence ID {0, 1,...,4095}, N_slot_symb
        l: OFDM symbol in time slot ns
        Mpn: pseudo-random sequence length
        BSID: base station number in the cellular network
        Ncp: cyclic prefix length
    output: Parameter detection:
        C: ????????????
    """
    #Initial conditions
    self.N_slot_symb = N_slot_symb
    self.n_s_f = n_s_f
    Nc = 1600
    self.PNS = np.zeros((len(l), Mpn, len(n_s_f)), dtype = int)
    for n_s in n_s_f:
        S = np.empty((len(l), Mpn))
        for i in range(len(l)):
            C = np.zeros(Mpn, dtype = int)#.tolist()
            Cinit = np.mod(((2**22)*np.floor(n_PRS_ID[n_s]/1024) + (2**10)*(N_slot_symb*n_s+l[i]+1)\
                            *(2*np.mod(n_PRS_ID[n_s],1024)+1) + np.mod(n_PRS_ID[n_s],1024)), 2**31)
            #Cinit=(2^10)*(7*(ns+1)+l+1)*(2*BSID+1)+2*BSID+Ncp
            #  31 Gold

            X1 = np.zeros((31), dtype = int)#.tolist()
            X1[0] = 1
            X2 = np.array([int(i) for i in np.binary_repr(int(Cinit), 31)[::-1]]) #.tolist()

            for n in np.arange(0, Mpn+Nc-31, 1):
            #X1[n+30 +1] = 11 #np.mod(X1[n+3+1] + X1[n+1], 2)
            #X2[n+30 +1] = 12 #np.mod(X2[n+3+1]+X2[n+2+1]+X2[n+1+1]+X2[n+1],2)
                X1 = np.append(X1, [np.mod((X1[n+3] + X1[n]), 2)])
                X2 = np.append(X2, [np.mod(X2[n+3] + X2[n+2] + X2[n+1] + X2[n], 2)])


        # alpha
            for n in np.arange(0, Mpn, 1):
                C[n] = np.mod(X1[n+Nc] + X2[n+Nc],2)

            S[i,:] = C

        self.PNS[:,:,n_s] = S
        return self

  def PRSSequence_5g(self, NPRB, MaxNDRB, lstart, l, nFFT, n_s_f, n_PRS_ID, K_PRS_comb, beta_PRS):
    self.nFFT = nFFT
#  PRSSequence  This program generates positioning reference signal sequence
#  Parameter Description:
#  NDRB：number of downlink resource blocks
#  NPRB：  number of positioning reference signal resource blocks
#  l：     OFDM symbol number in time slot ns
#  BSID：  base station number
#  ns：    time slot number
#  Ncp：   cyclic prefix length
#  Parameter detection
#l=l_PRS_start,l_PRS_start+1,...,l_PRS_start+L_PRS-1
#L_PRS=2，4，6，12L_PRS=2,4,6,12 ---- PRS resource size in the time slot L_PRS=2, 4, 6, 1
#k_PRS_offsetRE：={0，1，。。，K_PRS_comb-1} ---- k_PRS_offsetRE: offset={0,1,. . , K_PRS_comb-1
#k_PRS_offset = 0

    #l = np.arange(lstart,lstart+L_PRS-1,1)
    self.K_PRS_comb = K_PRS_comb
    k_prime_comb6 = [0,3,1,4,2,5,0,3,1,4,2,5]
    #k_prime_comb4 = [0,2,1,3,0,2,1,3,0,2,1,3]

    #PNS = PNSequence_5g(n_s_f, l,  Mpn, n_PRS_ID, N_slot_symb)

    self.PRS = np.empty((self.nFFT, self.N_slot_symb, len(self.n_s_f)), dtype=complex)
    for n_s in range(len(self.n_s_f)):
        PRS_temp = np.zeros((self.nFFT, self.N_slot_symb), dtype=complex)
        k_PRS_offset = np.mod(self.n_s_f[n_s], self.K_PRS_comb) #Frequency shift inside a symbol for multiple PRS repetition in different PRBs (Subcarriers)
        """if  k_PRS_offset == 0:
            k_PRS_offset += 1 #for frequency offset in the first PRS OFDM symbol --> +2 for first PRS ID and +4 for the second one
        if  k_PRS_offset == 1:
            k_PRS_offset += 2
        if  k_PRS_offset == 2:
            k_PRS_offset += 3"""

        for ll in range(len(l)):
            k1 = k_prime_comb6[l[ll]-lstart]
            c = self.PNS[ll,:,n_s]

            r  = np.zeros((NPRB + MaxNDRB), dtype = complex) #.tolist()
            X = np.zeros((nFFT), dtype = complex)

            for m in np.arange(0, 2*NPRB, 1):
                k  = m * K_PRS_comb + np.mod((k_PRS_offset + k1), K_PRS_comb)
                    # K_PRS_comb=2,4,6,12;   k_PRS_offset={0,1,...,K_PRS_comb-1}
                r[m] = (1/(np.sqrt(2)))*(1-2*c[2*m]) + 1j*(1/np.sqrt(2))*(1-2*c[2*m+1])
                X[k] = beta_PRS*r[m]

                PRS_temp[:,l[ll]] = X

        self.PRS[:,:, n_s] = PRS_temp

    return self

# ADD cyclic prefix at the begining of PRS signal
  def addCP(self, CP):
    CP = int(CP/2)
    #cp = self.PRS[-CP:,:,:]               # take the last CP samples ...
    #temp_ofdm_cp = np.hstack([cp, self.PRS])

    self.PRS_CP = np.empty((self.PRS.shape[0], self.PRS.shape[1], self.PRS.shape[2]), dtype=complex)
    for i in np.arange(self.PRS.shape[1]):
        for j in np.arange(self.PRS.shape[2]):
            cp = self.PRS[-CP:,i,j]
            PRS_temp = np.hstack([cp, self.PRS[:,i,j]])
            self.PRS_CP[:,i,j] = PRS_temp[:nFFT]
    #PRS_CP.append(P_C_temp)

    return self.PRS_CP  # ... and add them to the beginning


## PRSmaker parameters

In [6]:
lstart = 2
L_PRS = 6
K_PRS_comb = 6 #comb6 means {L_PRS = 6 --> number of filled symbols in time domain -1, K_PRS_comb = 6 --> factor of PRS repetition within one symbol}
N_slot_symb = 14
NPRB = [139] #number of positioning resource block --> 139*12*60 = 100,08 MHz BW #physical DL resource Blocks
#NPRB = [112, 139, 167] #corresponding to 80, 100, 120 MHz
MaxNDRB = 273
num_bs = 18
nFFT = 2048
beta_PRS = 1
lstart = 2
L_PRS = 6

l = [i for i in range(lstart, lstart+L_PRS-1, 1)]
n_s_f = [i for i in np.arange(0, num_bs, 1)] #slot number --> different slot index for differne BSs
n_PRS_ID = np.random.randint(0, 4095,len(n_s_f), dtype = int).tolist()

Mpn = [(MaxNDRB + NPRB)*2 for NPRB in NPRB]


K_PRS_comb = 6
CP = [(2048 - 12*NPRB) for NPRB in NPRB]

## making the PRSs


In [7]:
for ind, m in enumerate(Mpn):
  PRS = PRSmaker().PNSequence_5g(n_s_f, l,  m, n_PRS_ID, N_slot_symb)\
    .PRSSequence_5g(NPRB[0], MaxNDRB, lstart, l, nFFT, n_s_f, n_PRS_ID, K_PRS_comb, beta_PRS)\
    .addCP(CP[0])
PRS = PRS[np.newaxis,:]

print(f'PRS is repeated for BW {[NPRB*60*12/1000 for NPRB in NPRB]}MHz\nFinal PRS file length is {PRS.shape}')
print("Size of PRS:", sys.getsizeof(PRS), "bytes")

PRS is repeated for BW [100.08]MHz
Final PRS file length is (1, 2048, 14, 18)
Size of PRS: 160 bytes


# CIRmaker class object


In [8]:
class CIRmaker:
  def __init__(self, ds, ds_num, bs_id):
    self.Nrays = 20
    self.ds = ds
    self.ds_num = ds_num
    self.bs_id = bs_id
    self.power = self.ds[self.ds_num]['pows'][(self.bs_id)][()][()]
    self.phase = self.ds[self.ds_num]['phase'][(self.bs_id)][()][()]
    self.delay = self.ds[self.ds_num]['delays'][(self.bs_id)][()][()]

  def CIR(self):
    CIR = (1/np.sqrt(self.Nrays))*self.power*np.exp(1j*self.phase)
    return CIR

  def set_output(self):
    CIR = self.CIR()
    return CIR


# RX maker class object


In [9]:
class RXmaker:
    def __init__(self):

        self.nFFT = 2048
        self.SC_BW  = 60e3 #120e3 or 30e3
        self.Ts = 1/(self.SC_BW*self.nFFT)
        self.freq = np.fft.fftfreq(self.nFFT, self.Ts) / 1e6 #giga hz to mega hz

        self.RX = None

    def set_input(self, bs_id, SNRdB, PRS, CIR):
        self.bs_id = bs_id
        self.PRS = self.get_signal(PRS)
        self.CIR = CIR #self.get_signal(CIR)
        self.CFR = self.fft_transform()
        self.SNRdB = SNRdB


    def get_signal(self, signal):
        if len(signal.shape) == 4:
            return signal[:,:,2:7,self.bs_id] #self.flatten(signal[:,self.bs_id,:,:])

        if len(signal.shape) == 3:
            return signal[:,self.bs_id,:]

    def fft_transform(self):
        return np.fft.fft(self.CIR, n = self.nFFT, axis = 1)

    def flatten(self, signal):
        return signal.reshape((signal.shape[0], -1))

    def shape_alignment(self):
        #self.PRS = self.PRS[np.newaxis,:,:]
        #self.PRS = np.transpose(np.repeat(self.PRS[:, np.newaxis], repeats = 3, axis = 0), [1,2,0,3])
        self.PRS = np.transpose(self.PRS, [0,2,1])
        self.CFR = np.repeat(self.CFR[:,np.newaxis], repeats = 5, axis = 1)
        return self

    #@tf.function
    def RX_freq(self):
        self.shape_alignment()
        return np.multiply(self.PRS, self.CFR)

    def NoiseGenerator(self):
        np.random.seed(123)
        #mean = 0
        self.sigma_2 = self.signal_power * 10**(-self.SNRdB/10)
        return np.sqrt(self.sigma_2[:,:,np.newaxis]*0.5)*(np.random.randn(*self.CFR.shape)+1j*np.random.randn(*self.CFR.shape))
    #tf.complex(tf.random.normal(self.CFR.shape, mean, self.std), tf.random.normal(self.CFR.shape, mean, self.std))

    def RX_time(self):
        freq = self.RX_freq()
        RX = self.ifft_transform(freq)
        self.signal_power = np.mean(abs(RX), axis = 2)
        return RX

    def PRS_time(self):
        return np.squeeze(self.ifft_transform(self.PRS))

    def ifft_transform(self, signal):
        return np.fft.ifft(signal, n = self.nFFT, axis = 2)

    def set_output(self, mode):
        self.mode = mode
        if mode == 'Noisy':
            self.RX = self.RX_time() + self.NoiseGenerator()
        if mode == 'NotNoisy':
            self.RX = self.RX_time()
        #PRS_time = self.PRS_time()
        #self.get_info()
        return self.RX

    def get_info(self):
        print('Shape of input signals, PRS and CIR is {}, {}'.format(self.PRS.shape, self.CIR.shape))
        print(f'CFR, PRS alignment is performed,\nRepeated CIR for 5 symbols, and one dummy axis for multiplication\n \
        Final RX signal shape is {self.RX.shape}')
        if self.mode == 'Noisy':
            print('SNR = {:.2f}'.format(self.SNRdB))

# This block generate the RX signal for different base stations,
#bs_id = 11 previously is used for training the model and now, the robustness to predict LOS/NLOS signals from other BSs is the main goal
# fixed values according to previous analysis BW = 100, SYmbols = 5, SNR = 10, Dataset = 1, BS_id = [] for training

# Statistical Feature Extractor


In [10]:
class FeatureExtractor:
    def __init__(self):
        #self.delay = np.linspace(0, Ts, RX.shape[-1])
        self.SC = 60e3 #numerology of two, subcarrier spacing

    def set_input(self, RX, nFFT):
        self.nFFT = nFFT
        self.RX = RX
        self.Ts = 1/self.SC*nFFT
        BW = self.SC * self.nFFT
        self.delay = np.indices(self.RX.shape)[2] /BW
        self.pow = abs(self.RX)
        self.phase = np.angle(self.RX)


    def Average_power(self):
        return np.mean(self.pow, axis = 2)

    def MED(self):
        pow_ave = self.Average_power()
        return (1/pow_ave)*np.sum(self.delay*self.pow, axis = 2) # Mean excess delay

    def RMS_DS(self):
        pow_ave = self.Average_power()
        MED = self.MED()
        return (1/pow_ave)*(np.sum((np.transpose(np.array([(self.delay[:,:,i] - MED)**2 for i in np.arange(0,self.delay.shape[2],1)]), [1,2,0])*self.pow), axis = 2))

    def R_max(self):
        return np.max(np.apply_along_axis(np.sqrt, 2, self.pow), axis = 2)

    def Rise_time(self):
        r_max = self.R_max()
        sigma_n = 1 # standard deviation of thermal noise
        alpha = 6
        beta = 0.6
        r_t = np.apply_along_axis(np.sqrt, 2, self.pow)
        a_H = np.argmin(np.transpose(np.array([np.where( r_t[:,:,i] > r_max[:,:]*beta, r_t[:,:,i], r_max[:,:]*beta) for i in np.arange(0,self.pow.shape[2],1)]), [1,2,0]), axis = 2)
        a_L = np.argmin(np.transpose(np.array([np.where( r_t[:,:,i] > sigma_n*alpha, r_t[:,:,i], sigma_n*alpha) for i in np.arange(0,self.pow.shape[2],1)]), [1,2,0]), axis = 2)

        t_H = np.empty((a_H.shape[0],a_H.shape[1]))
        t_L = np.empty((a_L.shape[0],a_L.shape[1]))

        for i in np.arange(0,a_H.shape[0],1):
            for j in np.arange(0,a_H.shape[1],1):
                t_H[i,j] = self.delay[i,j, a_H[i,j]]

        for i in np.arange(0,a_L.shape[0],1):
            for j in np.arange(0,a_L.shape[1],1):
                t_L[i,j] = self.delay[i,j, a_L[i,j]]


        t_rise = t_H - t_L

        return t_rise

    def Kurtosis(self):
        r_t = np.apply_along_axis(np.sqrt, 2, self.pow)
        t_ave = np.sum(self.delay, axis = 2)
        mu_r = (1/t_ave)*np.sum(r_t, axis = 2)
        sigma_r = (1/self.delay.shape[2])*(np.sum((np.transpose(np.array([(r_t[:,:,i] - mu_r)**2 for i in np.arange(0,20,1)]), [1,2,0])), axis = 2))
        sigma_four = np.apply_along_axis(np.square, 1, sigma_r)
        kur = (1/(t_ave*sigma_four))*(np.sum((np.transpose(np.array([(r_t[:,:,i] - mu_r)**4 for i in np.arange(0,20,1)]), [1,2,0])), axis = 2))
        return kur

    def set_output(self):
        ave_pow = self.Average_power()
        MED = self.MED()
        RMS_DS = self.RMS_DS()
        r_max = self.R_max()
        Rise_time = self.Rise_time()
        Kurtosis = self.Kurtosis()
        return np.transpose(np.stack((ave_pow, MED, RMS_DS, r_max, Rise_time, Kurtosis)), [1,2,0])

    def info(self):
        print(f'the shape of maximum amplitude is {self.R_max().shape}')
        print(f'the shape of average power or received signal energy is {self.Average_power().shape}')
        print(f'the shape of mean excess delay (MED) or poweer delay profile (PDP) is {self.MED().shape}')
        print(f'the shape of RMS delay spread is {self.RMS_DS().shape}')
        print(f'the shape of rise time is {self.Rise_time().shape}')
        print(f'the shape of Kurtosis is {self.Kurtosis().shape}')

# Makinng Features for all BSs


In [14]:
#IMPORTANT!!! a unique PRS and CIR w.r.t each BS is selected inside RXmaker class object --> No need to declaire the PRS/CIR number
SNRdB = [0, 5, 10, 15, 20]

FE_out = [] # (7102,30,18) for all BSs
RX = RXmaker()
FE = FeatureExtractor()

bs_counts = 18
a = range(bs_counts)
last_value = list(a)[-1]
#with tf.device('/GPU:0'):
FE_BS = []
#with tpu_strategy.scope():
with tf.device('/GPU:0'):  
    for bs_iid in a: #change the range function for all TPs from 0 to 17
        CIR = CIRmaker(ds, ds_num, bs_iid).set_output()
        RX.set_input(bs_iid,SNRdB[2],PRS, CIR)
        RX_time_noisy = RX.set_output('Noisy')
        FE.set_input(RX_time_noisy, nFFT)
        FE_noisy = FE.set_output()
        FE_noisy = FE_noisy.reshape(FE_noisy.shape[0], -1) # there is 5 dataset and 5 symbols, when save as numpy array, the shapes become ambigeous, thus reshape the array here rather than inside ML code chunk
        ind = np.array([0, 1, 4, 5, 0+6, 1+6, 4+6, 5+6, 0+12, 1+12, 4+12, 5+12, 0+18, 1+18, 4+18, 5+18, 0+24, 1+24, 4+24, 5+24])
        FE_decorrelated = FE_noisy[:,ind]
        FE_out.append(FE_decorrelated)
        if bs_iid == last_value:
            RX.get_info()
            FE.info()
        del CIR
        del RX_time_noisy
        del FE_noisy
        del FE_decorrelated

X = np.array(FE_out)
print('The shape of input is {}'.format(X.shape))
print("Size of extracted features:", sys.getsizeof(X), "bytes")


2024-07-08 19:20:36.335307: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-07-08 19:20:36.335354: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-07-08 19:20:36.335364: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-07-08 19:20:36.335892: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-08 19:20:36.336436: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Shape of input signals, PRS and CIR is (1, 5, 2048), (7102, 20)
CFR, PRS alignment is performed,
Repeated CIR for 5 symbols, and one dummy axis for multiplication
         Final RX signal shape is (7102, 5, 2048)
SNR = 10.00
the shape of maximum amplitude is (7102, 5)
the shape of average power or received signal energy is (7102, 5)
the shape of mean excess delay (MED) or poweer delay profile (PDP) is (7102, 5)
the shape of RMS delay spread is (7102, 5)
the shape of rise time is (7102, 5)
the shape of Kurtosis is (7102, 5)
The shape of input is (18, 7102, 20)
Size of extracted features: 20453904 bytes


In [17]:
XX = X.reshape((-1,X.shape[2]))
YY = LOS.reshape(-1)
print(XX.shape, YY.shape)

(127836, 20) (127836,)


In [19]:
np.save('XX.npy', XX)
np.save('YY.npy', YY)